<a href="https://colab.research.google.com/github/maylovesart/maylovesart/blob/main/Political_Theory_Teaching_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import necessary libraries

In [25]:
import pandas as pd
import sqlite3
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 1. Data Preparation

In [26]:
print("1. Data Preparation")
df = pd.DataFrame({
    'topic': ['Social Contract Theory', 'Marxism', 'Liberalism', 'Feminism', 'Postcolonialism'],
    'subtopics': [
        'State of Nature;Social Contract;Legitimate Government',
        'Historical Materialism;Class Struggle;Dialectical Materialism',
        'Individual Rights;Free Market;Limited Government',
        'Gender Equality;Patriarchy;Intersectionality',
        'Cultural Hegemony;Subaltern;Orientalism'
    ],
    'content': [
        "Social contract theory examines the relationship between individuals and the state...",
        "Marxism is a socioeconomic analysis that critiques capitalism and class relations...",
        "Liberalism emphasizes individual rights, liberty, and equality before the law...",
        "Feminism advocates for women's rights and equality across various spheres...",
        "Postcolonialism studies the cultural legacy of colonialism and imperialism..."
    ]
})
print(df)
print("\n")


1. Data Preparation
                    topic                                          subtopics  \
0  Social Contract Theory  State of Nature;Social Contract;Legitimate Gov...   
1                 Marxism  Historical Materialism;Class Struggle;Dialecti...   
2              Liberalism   Individual Rights;Free Market;Limited Government   
3                Feminism       Gender Equality;Patriarchy;Intersectionality   
4         Postcolonialism            Cultural Hegemony;Subaltern;Orientalism   

                                             content  
0  Social contract theory examines the relationsh...  
1  Marxism is a socioeconomic analysis that criti...  
2  Liberalism emphasizes individual rights, liber...  
3  Feminism advocates for women's rights and equa...  
4  Postcolonialism studies the cultural legacy of...  




# 2. Database Creation

In [27]:
print("2. Database Creation")
conn = sqlite3.connect('political_theory_notes.db')
df.to_sql('notes', conn, if_exists='replace', index=False)
print("Database created successfully")
print("\n")

2. Database Creation
Database created successfully




# 3. Basic Search Function

In [28]:
print("3. Basic Search Function")
def search_notes(query):
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM notes WHERE content LIKE ?", ('%' + query + '%',))
    return cursor.fetchall()

print("Search results for 'social':")
print(search_notes('social'))
print("\n")

3. Basic Search Function
Search results for 'social':
[('Social Contract Theory', 'State of Nature;Social Contract;Legitimate Government', 'Social contract theory examines the relationship between individuals and the state...')]




# 4. Advanced Search with TF-IDF

In [29]:
print("4. Advanced Search with TF-IDF")
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['content'])

def advanced_search(query):
    query_vec = vectorizer.transform([query])
    similarity = cosine_similarity(query_vec, tfidf_matrix).flatten()
    related_docs_indices = similarity.argsort()[:-len(df)-1:-1]
    return df.iloc[related_docs_indices]

print("Advanced search results for 'equality':")
print(advanced_search('equality'))
print("\n")

4. Advanced Search with TF-IDF
Advanced search results for 'equality':
                    topic                                          subtopics  \
2              Liberalism   Individual Rights;Free Market;Limited Government   
3                Feminism       Gender Equality;Patriarchy;Intersectionality   
4         Postcolonialism            Cultural Hegemony;Subaltern;Orientalism   
1                 Marxism  Historical Materialism;Class Struggle;Dialecti...   
0  Social Contract Theory  State of Nature;Social Contract;Legitimate Gov...   

                                             content  
2  Liberalism emphasizes individual rights, liber...  
3  Feminism advocates for women's rights and equa...  
4  Postcolonialism studies the cultural legacy of...  
1  Marxism is a socioeconomic analysis that criti...  
0  Social contract theory examines the relationsh...  




# 5. Question Generation

In [30]:
print("5. Complex Question Generation")
def generate_complex_question():
    # Select two random topics
    topics = df['topic'].tolist()
    topic1, topic2 = random.sample(topics, 2)

    # Get subtopics for each topic
    subtopics1 = df[df['topic'] == topic1]['subtopics'].iloc[0].split(';')
    subtopics2 = df[df['topic'] == topic2]['subtopics'].iloc[0].split(';')

    # Select a random subtopic from each
    subtopic1 = random.choice(subtopics1)
    subtopic2 = random.choice(subtopics2)

    # Generate question
    question = f"Compare the concept of {subtopic1} in {topic1} with {subtopic2} in {topic2}. How do these ideas reflect the core principles of each theory?"

    return question

# Generate multiple complex questions
print("Generated complex comparison questions:")
for _ in range(3):
    print(generate_complex_question())
    print()

# Close the database connection
conn.close()

5. Complex Question Generation
Generated complex comparison questions:
Compare the concept of Intersectionality in Feminism with Subaltern in Postcolonialism. How do these ideas reflect the core principles of each theory?

Compare the concept of Patriarchy in Feminism with Historical Materialism in Marxism. How do these ideas reflect the core principles of each theory?

Compare the concept of Free Market in Liberalism with Patriarchy in Feminism. How do these ideas reflect the core principles of each theory?

